# ベイズ線形回帰の実装

## やること

1. 観測データセットの作成 \
今回はsin波にガウスノイズを付加したデータセットを15プロット作成 \
それを多項式： \
$y={\bf w}_0^{\mathrm{T}} \cdot {\bf x}^{3} + {\bf w}_1^{\mathrm{T}} \cdot {\bf x}^{2} + {\bf w}_2^{\mathrm{T}} \cdot {\bf x}^{1} + {\bf w}_3^{\mathrm{T}} \cdot 1$ \
で近似する
1. メインプログラム作成の流れ
    1. 観測データセットの作成
    1. 重み分布の更新プログラム
    1. 予測分布プログラム
    1. 結果を可視化するプログラム
    1. それらを呼び出すプログラム  

## 観測データセットの作成